In [1]:
import networkx as nx
import random
import numpy as np
import time
from itertools import combinations, groupby
from gensim.models import Word2Vec
from pecanpy import pecanpy
from scipy import sparse
import torch
import torch.optim as optim

# Create Dataset

In [2]:
def gnp_random_connected_graph(n: int, p: float) -> nx.Graph:
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted
    """
    edges = combinations(range(n), 2)
    G = nx.Graph()
    G.add_nodes_from(range(n))
    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < p:
                G.add_edge(*e)

    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = random.random()*100
    return G

def distance_sum(g):
    lengths = dict(nx.floyd_warshall(g, weight='weight'))
    
    total_sum=0
    
    for i in range(len(g.nodes)):
        node_lengths=lengths[i]
        node_sum=sum(np.array(list(node_lengths.values())))
        # print(node_sum)
        total_sum+=node_sum
    return total_sum

def get_node_embedding(graph,dimensions=128,walk_length=30, num_walks=200, workers=4,pfilename="test.edg"):
    print("Edges: "+str(len(graph.edges())))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    nodelist = list(graph.nodes)
    
    
    "already seen the q=5, lets try now q=2"
    
    ####Pecanpy
    nx.write_weighted_edgelist(graph, pfilename,delimiter='\t')
    
    # g=pecanpy.PreComp(p=1,q=5,workers=8,verbose=True)
    # g=pecanpy.SparseOTF(p=1,q=1,workers=8,verbose=True)
    
    g = pecanpy.SparseOTF(p=1, q=2, workers=8, verbose=True, extend=True, gamma=0)
    print("Extend=True")
    g.read_edg(pfilename, weighted=True ,directed=True)
    
    starttime2 = time.time()
    walks = g.simulate_walks(num_walks=10, walk_length=100)# use random walks to train embeddings
    model = Word2Vec(walks, vector_size=dimensions, window=5, min_count=0, sg=1, workers=8, epochs=10)
    endtime2 = time.time()
    duration = endtime2-starttime2
    
    print(f'Pecanpy {duration}')
    
    feature_matrix = (model.wv.vectors)
    
    feature_matrix_sparse = sparse.csr_matrix(feature_matrix)
    
    return feature_matrix_sparse


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    
    sparse_mx = sparse_mx.tocoo().astype(float)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    # FOR UNWEIGHTED, data only contains ones!!!!
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def adjacency_feature_centrality(graph: )

def get_adjacency_feature_centrality(list_graph, num_nodes,pfilename="test.edg"):
    
    """Returns adjacency matrices, their node sequences and centrality matrix."""
    
    num_graph = len(list_graph)
    # list_adj = list()
    # list_adj_t = list()
    cent_mat = np.zeros((1,num_graph),dtype=float)
    list_nodelist = list()
    list_feature_mat = list()
    list_edge_index=list()
    list_edge_weight=list()
    list_distance_sum=list()

    for ind,g in enumerate(list_graph):
        print("Generating Graph: "+str(ind))
        print("Max_Node: "+str(num_nodes))
        node_seq = list(g.nodes())
        
        starttime2 = time.time()
        # ebc=nx.betweenness_centrality(g, weight = 'weight')
        total_sum_new=distance_sum(g)
        # eff=total_sum_orig/total_sum_new
        
        endtime2 = time.time()
        duration = endtime2-starttime2
        
        print("EFF:"+str(duration))
        cent_mat[0, ind] = total_sum_new
          
        # adj_mat = np.zeros((num_nodes,num_nodes))
        # adj_mat_excerpt = get_adj_matrix(g).todense()
        # adj_mat[0:adj_mat_excerpt.shape[0], 0:adj_mat_excerpt.shape[1]] = adj_mat_excerpt
        # adj_mat = sparse.csr_matrix(adj_mat)
        

        
        ndim = 256
        feature_mat = np.zeros((num_nodes,ndim))
        feature_mat_excerpt = get_node_embedding(g, dimensions = ndim, walk_length = 50, num_walks = 50, workers = 12,pfilename=pfilename).todense()
        feature_mat[0:feature_mat_excerpt.shape[0]] = feature_mat_excerpt
        feature_mat = sparse.csr_matrix(feature_mat)

        # adj_mat = sparse_mx_to_torch_sparse_tensor(adj_mat)  
 
        feature_mat = sparse_mx_to_torch_sparse_tensor(feature_mat)
        
        # list_adj.append(adj_mat)
        list_edge_index.append(torch.tensor(list(g.edges)).t().contiguous())
        list_edge_weight.append(torch.tensor([g[u][v]['weight'] for u, v in g.edges]))

        
        list_nodelist.append(list(range(num_nodes)))
        list_feature_mat.append(feature_mat)
      
         
    # return list_adj,list_adj_t,list_nodelist,cent_mat,list_feature_mat
    return list_edge_index,list_edge_weight , cent_mat, list_feature_mat


def get_adjacency_centrality(list_graph, num_nodes):
    
    """Returns adjacency matrices, their node sequences and centrality matrix."""
    
    num_graph = len(list_graph)
    # list_adj = list()
    # list_adj_t = list()
    cent_mat = np.zeros((1,num_graph),dtype=float)
    list_nodelist = list()
    # list_feature_mat = list()
    list_edge_index=list()
    list_edge_weight=list()

    for ind,g in enumerate(list_graph):
        print("Generating Graph: "+str(ind))
        print("Max_Edge: "+str(num_nodes))
        node_seq = list(g.nodes())
        
        starttime2 = time.time()
        # ebc=nx.betweenness_centrality(g, weight = 'weight')
        total_sum_new=distance_sum(g)
        # eff=total_sum_orig/total_sum_new
        endtime2 = time.time()
        duration = endtime2-starttime2
        
        print("EFF:"+str(duration))
        cent_mat[0, ind] = total_sum_new
          
        # adj_mat = np.zeros((num_nodes,num_nodes))
        # adj_mat_excerpt = get_adj_matrix(g).todense()
        # adj_mat[0:adj_mat_excerpt.shape[0], 0:adj_mat_excerpt.shape[1]] = adj_mat_excerpt
        # adj_mat = sparse.csr_matrix(adj_mat)
        
        # adj_mat = sparse_mx_to_torch_sparse_tensor(adj_mat)  
        # list_adj.append(adj_mat)
        
        list_edge_index.append(torch.tensor(list(g.edges)).t().contiguous())
        list_edge_weight.append(torch.tensor([g[u][v]['weight'] for u, v in g.edges]))

        list_nodelist.append(list(range(num_nodes)))

    return list_edge_index,list_edge_weight, cent_mat#, list_feature_mat

In [20]:
def modify_graph(base_graph: nx.Graph, k: int) -> nx.Graph:
    """
    Remove k edges from a graph while maintaining connectedness
    """
    modified_graph = base_graph.copy()

    while True:
        test_graph = modified_graph.copy()
        edges_to_delete = random.sample(list(modified_graph.edges()), k)
        edges_to_delete.sort(reverse=True)

        for u, v in edges_to_delete:
            test_graph.remove_edge(u, v)
        
        if nx.is_connected(test_graph):
            for u, v in edges_to_delete:
                modified_graph.edges[u, v]['weight'] = 1000000000
            break
    
    return modified_graph


# Create N_GRAPHS training graphs
N_GRAPHS = 10
MAX_NODES = 100
PROB_EDGE = 1.2/(100-1)
base_graphs: 'list[nx.Graph]' = [gnp_random_connected_graph(MAX_NODES, PROB_EDGE) for _ in range(N_GRAPHS)]

# Create 1 modified graph for each
modified_graphs = [modify_graph(base_graph, k=1) for base_graph in base_graphs]

'''
Use this to draw unmodified + modified
'''

# G = base_graphs[0]
# labels = nx.get_edge_attributes(G, 'weight')
# pos = nx.spring_layout(G, k=0.15, iterations=20)
# nx.draw(G, pos, with_labels=True)
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

# G = modified_graphs[0]
# labels = nx.get_edge_attributes(G, 'weight')
# # pos = nx.spring_layout(G, k=0.15, iterations=20)
# nx.draw(G, pos, with_labels=True)
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

'\nUse this to draw unmodified + modified\n'

In [21]:
import os.path as osp

import torch
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx, from_networkx

class PairData(Data):
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index_base':
            return self.num_nodes
        if key == 'edge_index_modified':
            return self.num_nodes
        return super().__inc__(key, value, *args, **kwargs)
    
torch_base: 'list[Data]' = list(map(from_networkx, base_graphs))
torch_modified: 'list[Data]' = list(map(from_networkx, modified_graphs))

torch_base_edge_index, torch_base_weights = list(zip(*[(g.edge_index, g['weight']) for g in torch_base]))
torch_modified_edge_index, torch_modified_weights = list(zip(*[(g.edge_index, g['weight']) for g in torch_modified]))

pair_data = list(zip(torch_base_edge_index, torch_base_weights, torch_modified_edge_index, torch_modified_weights))

data_list = [PairData(edge_index_base=edge_index_base, 
                      weight_base=weight_base, 
                      edge_index_modified=edge_index_modified,
                      weight_modified=weight_modified,
                      num_nodes=MAX_NODES)
                for edge_index_base, weight_base, edge_index_modified, weight_modified in pair_data
            ]

TRAIN_TEST_SPLIT = 0.8
BATCH_SIZE = 4
split_index = int(TRAIN_TEST_SPLIT * N_GRAPHS)
train_dataset = data_list[:split_index]
test_dataset = data_list[split_index:]

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [83]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class Graph_Diff_Reg(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim,dropout):
        super(Graph_Diff_Reg, self).__init__()
        self.dropout = dropout

        self.gcn1 = GCNConv(input_dim, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)

        self.mlp = nn.Sequential(
            nn.Linear(int(hidden_dim), hidden_dim),
            # nn.ReLU(),
            # nn.Tanh(),
            # nn.Sigmoid(),
            nn.Dropout(p=self.dropout),
            nn.Linear(hidden_dim, int(hidden_dim/2)),
            nn.Dropout(p=self.dropout),
            nn.Linear(int(hidden_dim/2), int(hidden_dim/4)),
            nn.Dropout(p=self.dropout),
            # # nn.Linear(int(hidden_dim/4), int(hidden_dim/8)),
            # # nn.ReLU(),
            # # nn.Tanh(),
            # nn.Dropout(p=self.dropout),
            nn.Linear(int(hidden_dim/4),1)
        )


    def forward(self,edge_index1,edge_weight1,edge_index2,edge_weight2,fm,batch_tensor):#,adj,adj_orig):
        # x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN layer
        x11 = self.gcn1(fm, edge_index1,edge_weight1)
        # print(sum())
        # x11 = F.relu(x11)

        # First GCN layer
        # with torch.no_grad():
        x12 = self.gcn1(fm, edge_index2,edge_weight2)
        # x12 = F.relu(x12)
        
        
        ###Secong GCN layer
        x21 = self.gcn2(x11, edge_index1,edge_weight1)
        # x21 = F.relu(x21)

        # First GCN layer
        # with torch.no_grad():
        x22 = self.gcn2(x12 , edge_index2,edge_weight2)
        # x22 = F.relu(x22)
        
        
        # x1=x11+x21+x31
        # x2=x12+x22+x32
        
        x1=x12-x11
        x2=x22-x21
        # x3=x32-x31
        # x4=x42-x41
        
        x=x1*x2#*x3#*x4
        
        
        # s = self.diff_pool(x, edge_index1, edge_weight1)

        # # Updated node representations after DiffPool
        # x = s.x
        
        
        
        
        # x=x1-x2
        
        # print(x)
        # print(x.shape)
        # Graph pooling
        
        # from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
        # x = torch.cat([gmp(x, batch_tensor), 
        #             gap(x, batch_tensor)], dim=1)
        
        x = global_mean_pool(x,batch_tensor)
        
        # MLP layers
        x = self.mlp(x)

        return x

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [86]:
from torch import optim

num_epoch = 50

hidden = 256
n_features = 256
dropout_val = 0.4
learning_rate = 0.0001

model = Graph_Diff_Reg(input_dim=n_features, hidden_dim=hidden, output_dim=1, dropout=dropout_val)
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.00005/10)

In [ ]:
def train(model: Graph_Diff_Reg, optimizer: optim.Adam):
    model.train()
    
    for data in train_loader:
        out = model()

In [ ]:
n_nodes=500
prob_edge = 1.2/(100-1)

BASE_GRAPH = gnp_random_connected_graph(n_nodes, prob_edge)
MAX_MODEL_NUM = len(BASE_GRAPH.nodes)

X_train_base = [BASE_GRAPH.to_directed()]
X_test_base =  [BASE_GRAPH.to_directed()]